In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, '../')

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

## Loading face detector

In [5]:
import dlib

In [6]:
# Create a HOG face detector using the built-in dlib class
face_detector = dlib.get_frontal_face_detector()
# Using cnns
# face_detector = dlib.cnn_face_detection_model_v1('../models/mmod_human_face_detector.dat')

## Loading alignment model

In [7]:
from cpe775.align import AlignDlib

In [8]:
align = AlignDlib()

## Loading embedding model

In [9]:
from cpe775.networks.openface import OpenFace
from cpe775.face_detector import FaceEmbedding

In [10]:
embedding = FaceEmbedding('../models/openface/nn4.small2.v1.pth')

## Loading landmarks regressor

In [11]:
from cpe775.networks.resnet import resnet18
from cpe775.face_detector import LandmarksDetector

In [12]:
net = resnet18(in_shape=(-1, 1, 256, 256), out_shape=(-1, 68, 2))
landmarks_detector = LandmarksDetector(net, '../models/landmarks_detector.pth')

## Loading classification model

In [13]:
from cpe775.face_detector import FaceClassify

In [14]:
classify = FaceClassify('../models/face_classification_svm.pkl')

## Loading mapping

## Using the webcam
Adapted from: https://github.com/ageitgey/face_recognition

In [15]:
import cv2

# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.


In [20]:
# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Initialize some variables
face_locations = []
face_names = []
face_landmarks = []
face_probas = []
process_this_frame = True
scale = 0.25
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
    
    # Only process every other frame of video to save time
    if process_this_frame:
        
        face_locations = []
        face_names = []
        face_probas = []
        face_landmarks = []
        
        # Find all the faces and face encodings in the current frame of video
        detected_faces = face_detector(small_frame, 2)
        
        # Loop through each face we found in the image
        for i, face_rect in enumerate(detected_faces):

            # Find the landmarks given the image and the bounding boxes
            landmarks, rect = landmarks_detector.from_rect(small_frame, face_rect)
            
            face_locations.append(rect)
            
            face_landmarks.append(landmarks)
        
            # Align and resize the face given the landmarks
            thumbnail = align(small_frame, landmarks)
            
            # Extract the embedding from the aligned face 
            embed = embedding(thumbnail)
            
            # Classify
            name, prob = classify(embed)
            
            face_probas.append(prob)
            
            if prob > 0.5:
                face_names.append(name)
            else:
                face_names.append('unknown')

    process_this_frame = not process_this_frame


    # Display the results
    for (left, top, right, bottom), name, prob, landmarks in zip(face_locations, face_names, face_probas, face_landmarks):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top = int(top/scale)
        right = int(right/scale)
        bottom = int(bottom/scale)
        left = int(left/scale)

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, '{}, {:.2f}%'.format(name, prob*100), (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)
        
        # loop over the (x, y)-coordinates for the facial landmarks and draw them on the image
        for (x, y) in landmarks:
            cv2.circle(frame, (int(x/scale), int(y/scale)), 1, (0, 0, 255), -1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()